In [7]:
# !pip install -r requirements.txt
# !pip install git+https://github.com/PyTorchLightning/pytorch-lightning

In [4]:
# imports
import transformers
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import pandas as pd
import numpy as np

import torch.nn.functional as F
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint

import math
import random
import re
import argparse

In [5]:
from transformers import (
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Trainer,
    TrainingArguments,
)

# 모델, 토크나이저 정의

In [6]:
from kobart_transformers import get_kobart_model, get_kobart_tokenizer
from transformers import PreTrainedTokenizerFast
from transformers import BartForSequenceClassification
# tokenizer = get_kobart_tokenizer()
# model = get_kobart_model()
tokenizer = AutoTokenizer.from_pretrained("hyunwoongko/kobart")
model = AutoModelForSeq2SeqLM.from_pretrained("hyunwoongko/kobart")

# 데이터 로드 및 병합

In [8]:
import nlp
data = pd.read_excel("bonobono.xlsx")
dataset = data.iloc[:,2:4]
dataset = dataset.dropna(axis=0,how='any')

In [9]:
data1 = pd.read_excel("sea1.xlsx")
dataset1 = data1.iloc[:,2:4]
dataset1 = dataset1.dropna(axis=0,how='any')

In [10]:
data2 = pd.read_excel("sea2.xlsx")
dataset2 = data2.iloc[:,2:4]
dataset2 = dataset2.dropna(axis=0,how='any')

In [ ]:
# data3 = pd.read_excel("chatbot.xlsx")
# dataset3 = data3.iloc[:,1:3]
# dataset3 = dataset3.dropna(axis=0,how='any')

# 데이터 이상치가 많음

In [11]:
dataset = dataset.append((dataset1,dataset2))

In [12]:
# dataset.info()

In [13]:
train_dataset = nlp.Dataset.from_pandas(dataset[:1100])
val_dataset = nlp.Dataset.from_pandas(dataset[1100:])

## MAX_LENGTH 구하기

In [15]:
# train_dataset
print('존댓말의 평균 길이 :',sum(map(len, dataset["존댓말A"]))/len(dataset["존댓말A"]))

존댓말의 평균 길이 : 28.98505338078292


In [26]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))
max_len = 85
below_threshold_len(max_len, dataset['존댓말A'])

전체 샘플 중 길이가 85 이하인 샘플의 비율: 95.16014234875445


## Convert

In [27]:
def map_to_encoder_decoder_inputs(batch):
    inputs = tokenizer(batch["반말A"], padding="max_length", truncation=True, max_length=85) # 반말
    outputs = tokenizer(batch["존댓말A"], padding="max_length", truncation=True, max_length=85) # 존댓말

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    batch["labels"] = outputs.input_ids
    # batch["labels"] = [
    #     [-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]
    # ]
    
    # print("inputs:",inputs)
    # print("outputs:",outputs)
    assert all([len(x) == 85 for x in inputs.input_ids])
    assert all([len(x) == 85 for x in outputs.input_ids])

    return batch

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
      "rouge2_precision": round(rouge_output.precision, 4),
      "rouge2_recall": round(rouge_output.recall, 4),
      "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [28]:
batch_size = 16

train_dataset = train_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=batch_size, remove_columns=["반말A", "존댓말A"],
)
train_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask","labels"],
)

val_dataset = val_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=batch_size, remove_columns=["반말A", "존댓말A"],
)
val_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask","labels"],
)

In [29]:
model.to('cuda')

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768, padding_idx=3)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
   

In [ ]:
# !nvidia-smi

## 학습

In [30]:
# base_dir = 'content/'
# checkpoint = ModelCheckpoint(filepath=base_dir + 'checkpoint_files_1/')

training_args = TrainingArguments(
    output_dir="./",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # evaluate_during_training=True,
    do_train=True,
    # do_eval=True,
    logging_steps=5,
#     save_steps=5,
    eval_steps=20,
    overwrite_output_dir=True,
    warmup_steps=50,
    save_total_limit=1,
    num_train_epochs=5,
    # checkpoint_callback = checkpoint,
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)



In [31]:
trainer.train()

Step,Training Loss
5,19.555000
10,15.543000
15,11.642000
20,9.211700
25,6.971100
30,5.278200
35,4.365100
40,3.559000
45,2.913900
50,2.151500


TrainOutput(global_step=345, training_loss=1.3107514503209488, metrics={'train_runtime': 254.0838, 'train_samples_per_second': 1.358, 'total_flos': 347427210240000, 'epoch': 5.0})

In [35]:
inputs = tokenizer.encode("맞아, 좀 편해지고 싶어", return_tensors='pt')
model.to('cpu')
pred = model.generate(inputs)
tokenizer.decode(pred[0])

'<usr>맞아요, 편해지고 싶어요.<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [36]:
torch.save(model.state_dict(), 'kobart')

In [3]:
model.load_state_dict(torch.load('kobart'))
model.eval()
type(model)

In [45]:
class Styleformer():

    def __init__(self,  style=0):
        from transformers import AutoTokenizer
        from transformers import AutoModelForSeq2SeqLM
#         from styleformer import Adequacy

        self.style = style
        self.adequacy = Adequacy()
        self.model_loaded = False

        if self.style == 0:
            self.ctf_tokenizer = AutoTokenizer.from_pretrained("hyunwoongko/kobart")
            model.load_state_dict(torch.load('kobart'))
            self.ctf_model = model

            
            print("Casual to Formal model loaded...")
            self.model_loaded = True
        else:
            print("Only CTF, FTC, ATP and PTA are supported in the pre-release...stay tuned")

    def transfer(self, input_sentence, inference_on=0, quality_filter=0.95, max_candidates=5):
        if self.model_loaded:
            if inference_on == 0:
                device = "cpu"
            elif inference_on == 1:
                device = "cuda:0"  
            else:  
                device = "cpu"
                print("Onnx + Quantisation is not supported in the pre-release...stay tuned.")

            if self.style == 0:
                output_sentence = self._casual_to_formal(input_sentence, device, quality_filter, max_candidates)
                return output_sentence     
        else:
            print("Models aren't loaded for this style, please use the right style during init")  


    def _casual_to_formal(self, input_sentence, device, quality_filter, max_candidates):
        ctf_prefix = "transfer Casual to Formal: "
        src_sentence = input_sentence
        input_sentence = ctf_prefix + input_sentence
        input_ids = self.ctf_tokenizer.encode(input_sentence, return_tensors='pt')
        self.ctf_model = self.ctf_model.to(device)
        input_ids = input_ids.to(device)

        
        preds = self.ctf_model.generate(
            input_ids,
            do_sample=True, 
            max_length=32, 
            top_k=50, 
            top_p=0.95, 
            early_stopping=True,
            num_return_sequences=max_candidates)
        gen_sentences = set()
        for pred in preds:
            gen_sentences.add(self.ctf_tokenizer.decode(pred, skip_special_tokens=True).strip())
        print("gensentences",gen_sentences)
        adequacy_scored_phrases = self.adequacy.score(src_sentence, list(gen_sentences), quality_filter, device)
        ranked_sentences = sorted(adequacy_scored_phrases.items(), key = lambda x:x[1], reverse=True)
        print("ranked",ranked_sentences)
        if len(ranked_sentences) > 0:
            return ranked_sentences[0][0]
        else:
            return None


     
        return self.pta_tokenizer.decode(preds[0], skip_special_tokens=True).strip()      

    


In [47]:
# 모델변경 및 그에 따른 코드 변경 필요

class Adequacy():

  def __init__(self, model_tag='prithivida/parrot_adequacy_on_BART'):
    from transformers import AutoModelForSequenceClassification, AutoTokenizer
    from transformers import PreTrainedTokenizerFast
    # self.nli_model = BartModel.from_pretrained(model_tag)
    # self.tokenizer = PreTrainedTokenizerFast.from_pretrained(model_tag)
    
    self.nli_model = AutoModelForSequenceClassification.from_pretrained(model_tag)
    self.tokenizer = AutoTokenizer.from_pretrained(model_tag)
    
  def filter(self, input_phrase, para_phrases, adequacy_threshold, device="cpu"):
      top_adequacy_phrases = []
      for para_phrase in para_phrases:
        x = self.tokenizer.encode(input_phrase, para_phrase, return_tensors='pt',truncation_strategy='only_first')
        self.nli_model = self.nli_model.to(device)
        logits = self.nli_model(x.to(device))[0]
        # we throw away "neutral" (dim 1) and take the probability of "entailment" (2) as the adequacy score
        entail_contradiction_logits = logits[:,[0,2]]
        probs = entail_contradiction_logits.softmax(dim=1)
        prob_label_is_true = probs[:,1]
        adequacy_score = prob_label_is_true[0].item()
        if adequacy_score >= adequacy_threshold:
            top_adequacy_phrases.append(para_phrase)
      return top_adequacy_phrases

  def score(self, input_phrase, para_phrases, adequacy_threshold, device="cpu"):
      adequacy_scores = {}
      for para_phrase in para_phrases:
        x = self.tokenizer.encode(input_phrase, para_phrase, return_tensors='pt',truncation_strategy='only_first')
        self.nli_model = self.nli_model.to(device)
        logits = self.nli_model(x.to(device))[0]
        # we throw away "neutral" (dim 1) and take the probability of "entailment" (2) as the adequacy score
        entail_contradiction_logits = logits[:,[0,2]]
        probs = entail_contradiction_logits.softmax(dim=1)

        prob_label_is_true = probs[:,1]
        print("prob_label_is_true : ",prob_label_is_true )
        adequacy_score = prob_label_is_true[0].item()
        if adequacy_score >= adequacy_threshold:
          adequacy_scores[para_phrase] = adequacy_score

      return adequacy_scores      

## 테스트

In [4]:
# from styleformer import Styleformer
import torch
import warnings
warnings.filterwarnings("ignore")

'''
#uncomment for re-producability
def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(1234)
'''

sf = Styleformer(style = 0)

source_sentences = [
"맞아, 좀 편해지고 싶어",

]

for source_sentence in source_sentences:
    target_sentence = sf.transfer(source_sentence)
    print("-" *100)
    print("[Casual] ", source_sentence)
    print("-" *100)
    if target_sentence is not None:
        print("[Formal] ",target_sentence)
        print()
    else:
        print("No good quality transfers available !")
